Source for the GloVe embedding code: https://towardsdatascience.com/sentiment-analysis-using-lstm-and-glove-embeddings-99223a87fe8e

In [1]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [2]:
import numpy as np
import tensorflow as tf
import os 
import pandas as pd
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sn
import pickle

%matplotlib inline

#Import module to split the datasets
from sklearn.model_selection import train_test_split
# Import modules to evaluate the metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE

from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D, Conv1D, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D, LSTM,Conv1D, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import SGD


In [3]:
# Create the confussion matrix
def plot_confussion_matrix(y_test, y_pred):
    ''' Plot the confussion matrix for the target labels and predictions '''
    cm = confusion_matrix(y_test, y_pred)

    # Create a dataframe with the confussion matrix values
    df_cm = pd.DataFrame(cm, range(cm.shape[0]),
                  range(cm.shape[1]))
    #plt.figure(figsize = (10,7))
    # Plot the confussion matrix
    sn.set(font_scale=1.4) #for label size
    sn.heatmap(df_cm, annot=True,fmt='.0f',annot_kws={"size": 10})# font size
    plt.show()
    
# ROC Curve
# plot no skill
# Calculate the points in the ROC curve
def plot_roc_curve(y_test, y_pred):
    ''' Plot the ROC curve for the target labels and predictions'''
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
    roc_auc= auc(fpr,tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.chdir('/content/drive/MyDrive/Thesis/Data/Filled Datasets/Second dataset (Kaggle)/Preprocessed/')
train = pd.read_csv("Constraint_Train_fullprep.csv")
test = pd.read_csv("Constraint_Test_fullprep_with_labels.csv")
val = pd.read_csv('Constraint_Val_fullprep.csv')
print('train',train.head(n=3))
print('test',test.head(n=3))
print('val',val.head(n=3))
                  

train                                                tweet  label
0  cdc currently reports deaths general discrepan...      0
1  states reported deaths small rise last tuesday...      0
2  politically correct woman almost uses pandemic...      1
test    id                                              tweet  label
0   1  daily update published states reported k tests...      0
1   2                                 alfalfa cure covid      1
2   3      president trump asked would catch coronavirus      1
val                                                tweet  label
0  chinese converting islam realising muslim affe...      1
1  people diamond princess cruise ship initially ...      1
2       covid caused bacterium virus treated aspirin      1


In [6]:
X_train= train['tweet']
y_train= train['label']
X_test = test['tweet']
y_test = test['label']
X_val = val['tweet']
y_val = val['label']

In [7]:
#we need the next code because the one-hot encoding below otherwise doesn't work
for i in range(len(X_train)): 
    X_train.iloc[i] = str(X_train.iloc[i])

for i in range(len(X_val)): 
    X_val.iloc[i] = str(X_val.iloc[i])

for i in range(len(X_test)): 
    X_test.iloc[i] = str(X_test.iloc[i])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


tokenize the words 

In [8]:
X_train_list = []

for i in range(len(X_train)): 
  X_train_list.append(X_train[i])

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train_list)

words_to_index = tokenizer.word_index

In [9]:
X_val_list = []

for i in range(len(X_val)): 
  X_val_list.append(X_val[i])

In [10]:
X_test_list = []

for i in range(len(X_test)): 
  X_test_list.append(X_test[i])

function to read glove vector file, which returns dictionary that maps the words to their embeddings

In [11]:
import os
os.chdir('/content/drive/MyDrive/Thesis/Data/Embedding files/')
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [12]:
word_to_vec_map = read_glove_vector('glove.twitter.27B.100d.txt')

In [13]:
maxLen = 100

embedding matrix

In [14]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['virus'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index-1, :] = embedding_vector

In [21]:
vocab_len

10142

In [15]:
emb_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.21254 , -0.28039 , -0.86803 , ...,  0.083941, -0.089397,
         0.18572 ],
       [ 0.066068,  0.12279 , -1.1359  , ...,  1.0283  , -0.15395 ,
         0.54496 ],
       ...,
       [-0.042443,  0.27053 , -0.46525 , ..., -0.87101 , -0.3004  ,
         0.61371 ],
       [ 0.051175, -0.13945 ,  0.49631 , ..., -0.15366 , -0.37818 ,
        -0.086145],
       [-1.1537  , -0.95074 , -0.12871 , ...,  0.059921, -0.1734  ,
         0.71542 ]])

In [16]:
X_train_indices = tokenizer.texts_to_sequences(X_train_list)
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(6420, 100)

In [17]:
X_val_indices = tokenizer.texts_to_sequences(X_val_list)
X_val_indices = pad_sequences(X_val_indices, maxlen=maxLen, padding='post')
X_val_indices.shape

(2140, 100)

In [18]:
X_test_indices = tokenizer.texts_to_sequences(X_test_list)
X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')
X_test_indices.shape

(2139, 100)

**Oversampling**

In [19]:
smote = SMOTE()
X_train_indices, y_train = smote.fit_resample(X_train_indices,y_train)

In [22]:
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D, Conv1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding

model = Sequential([
    Embedding(vocab_len, 8, input_length=100),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 8)            40000     
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           5248      
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 46,549
Trainable params: 46,549
Non-trainable params: 0
____________________________________________________

In [24]:
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
checkpoint_path = "/home/lowie/data/rnn_check/"

monitor = 'val_precision'
keras_callbacks   = [
      EarlyStopping(monitor=monitor, patience=30, mode='max', min_delta=0.0001),
      ModelCheckpoint(checkpoint_path, monitor=monitor, save_best_only=True, mode='max')
]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tensorflow.keras.metrics.Precision()])

In [25]:
history = model.fit(X_train_indices, y_train, epochs=500, validation_data=(X_val_indices, y_val),callbacks=keras_callbacks)

Epoch 1/500
210/210 [==============================] - 5s 18ms/step - loss: 0.5024 - precision: 0.7671 - val_loss: 0.2621 - val_precision: 0.8857
Epoch 2/500
210/210 [==============================] - 3s 15ms/step - loss: 0.2192 - precision: 0.9147 - val_loss: 0.2133 - val_precision: 0.9157
Epoch 3/500
210/210 [==============================] - 3s 16ms/step - loss: 0.1300 - precision: 0.9522 - val_loss: 0.2269 - val_precision: 0.8986
Epoch 4/500
210/210 [==============================] - 3s 15ms/step - loss: 0.0763 - precision: 0.9750 - val_loss: 0.2532 - val_precision: 0.9038
Epoch 5/500
210/210 [==============================] - 4s 18ms/step - loss: 0.0433 - precision: 0.9872 - val_loss: 0.2911 - val_precision: 0.9127
Epoch 6/500
210/210 [==============================] - 3s 12ms/step - loss: 0.0233 - precision: 0.9929 - val_loss: 0.3204 - val_precision: 0.9088
Epoch 7/500
210/210 [==============================] - 3s 12ms/step - loss: 0.0137 - precision: 0.9967 - val_loss: 0.3581 - 

In [27]:
loss, accuracy = model.evaluate(X_test_indices,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))

67/67 [==============================] - 1s 8ms/step - loss: 0.8107 - precision: 0.8776
Testing Accuracy is 87.7629041671753 


In [28]:
y_pred = model.predict(X_test_indices)
y_pred = y_pred.round()

print(metrics.classification_report(y_test, y_pred,  digits=5))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0    0.90668   0.88561   0.89602      1119
           1    0.87763   0.90000   0.88867      1020

    accuracy                        0.89247      2139
   macro avg    0.89215   0.89281   0.89235      2139
weighted avg    0.89283   0.89247   0.89252      2139

[[991 128]
 [102 918]]


**SAVE THE MODEL**

In [29]:
os.chdir('/content/drive/MyDrive/Thesis/Models/Replicated models /Simple models /CNN/')
model.save('CNN_GLOVE.h5')

In [30]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("CNN_GLOVE.json", "w") as json_file:
    json_file.write(model_json)

In [31]:
from keras.models import model_from_json

# load json and create model
json_file = open('CNN_GLOVE.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("CNN_GLOVE.h5")
print("Loaded CNN_GLOVE from disk")

Loaded CNN_GLOVE from disk
